# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import numpy as np
import pandas as pd
import re

sentiment = pd.read_csv('Sentiment140.csv')
sentiment_sample = sentiment.sample(n=20000, random_state=1)

In [2]:
sentiment_sample.shape

(20000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
import nltk
nltk.download('punkt')

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/eleonore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eleonore/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eleonore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# your code here

# Python program to demonstrate  
# use of class method and static method. 
  
class text_processed: 
    def __init__(self, s): 
        self.s = s
        
    def clean_up(s):
        """Cleans up numbers, URLs, and special characters from a string."""
        list_to_sub=['\d','http?:\/\/.*[\d]*','@','\W']
    
        """Args: s: The string to be cleaned up."""
        for exp in list_to_sub:
            to_token= re.sub(exp, r' ', s).lower()

        """ Returns: A string that has been cleaned up."""
        return(to_token)

    def tokenize(to_token):
        """
        Tokenize a string."""

        """
        Args:
            s: String to be tokenized."""
        to_list = nltk.word_tokenize(to_token)

        """
        Returns:
            A list of words as the result of tokenization.
        """
        return(to_list)

    def stem_and_lemmatize(to_list):
        """Perform stemming and lemmatization on a list of words.

        Args: l: A list of strings.

        Returns: A list of strings after being stemmed and lemmatized."""   
        
        ps = PorterStemmer()
        to_stem = [ps.stem(word) for word in to_list]

        lemmatizer = WordNetLemmatizer()
        to_lemm = [lemmatizer.lemmatize(word) for word in to_stem]
        #print(to_lemm)
        return(to_lemm)

    def remove_stopwords(to_lemm):
        """ Remove English stopwords from a list of strings.

        Args: l: A list of strings.

        Returns:  A list of strings after stop words are removed. """

        filtered_sentence=[]
        stop_words = set(stopwords.words('english'))
        for w in to_lemm: 
            if w not in stop_words: 
                filtered_sentence.append(w)
        #print(filtered_sentence)
        return(filtered_sentence)
    
sentiment_sample = sentiment_sample.assign(text_processed=sentiment_sample['text'].map(lambda s: text_processed.remove_stopwords(text_processed.stem_and_lemmatize(text_processed.tokenize(text_processed.clean_up(s))))))


In [5]:
sentiment_sample[['text_processed']].head()

,text_processed
514293,"[miss, nikki, nu, nu, alreadi, alway, need, th..."
142282,"[dream, last, night, rememb, sign, clearli, to..."
403727,"[girlyghost, ohh, poor, sickli, hug, hope, fee..."
649503,[rain]
610789,"[misskeribabi, wish, wa, la, right]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [6]:
# your code here
from nltk.probability import FreqDist

to_freq=[word for sentence in sentiment_sample['text_processed'] for word in sentence]
fdist = FreqDist(word for word in to_freq)
 
for key,val in fdist.items():
    print (str(key) + ':' + str(val))

miss:753
nikki:6
nu:9
alreadi:189
alway:187
need:571
thank:731
u:854
xxx:49
dream:87
last:454
night:542
rememb:85
sign:58
clearli:4
told:42
get:1385
job:143
remem:1
said:127
girlyghost:1
ohh:21
poor:91
sickli:1
hug:70
hope:539
feel:602
littl:233
better:288
soon:202
rain:218
misskeribabi:1
wish:426
wa:1334
la:54
right:371
nala:1
olowalu:1
still:556
ha:490
full:67
tummi:34
bread:15
basket:3
2:546
http:881
app:33
facebook:58
com:631
dogbook:3
profil:25
view:31
6285546:1
macintom:1
site:61
seem:118
want:750
load:42
must:121
lot:181
hit:79
time:792
sleep:375
hav:16
actual:162
work:1026
tmrw:9
supercoolkp:1
oxford:1
month:111
cup:32
tea:42
fruit:9
bagel:4
go:1718
turn:90
day:1290
around:103
finish:162
play:239
asshol:7
first:218
2nd:29
place:88
second:49
1st:41
heh:16
final:232
got:893
green:31
bad:330
dog:84
kept:14
tryna:7
mix:25
guess:147
snifflin:1
sneezin:1
babimattivi:1
nhaaaiiiiiiiiiii:1
rendi:1
ao:2
twitter:463
fim:1
de:25
agosto:1
na:510
área:1
hein:1
kitti:17
make:542
lost:163
nurs

bam:2
routin:6
tonit:17
zachhump:1
wassup:1
chillllllllll:1
practic:34
winni:1
michael:11
jessica:4
mandi:8
tessah:1
bleumuz:1
jenmaselli:1
sneak:5
rave:3
strang:12
emo:6
lurk:1
jusadbella:1
stand:33
franki:1
solid:6
woke:106
45:21
ate:39
frozen:7
milk:10
stardust:1
confess:1
behalf:1
lebron:3
laker:20
nba:5
disc:3
tree:21
linehan:1
tweeter:13
upper:4
hangov:29
function:11
dsi:3
user:13
gener:15
segment:3
ad:44
collag:1
smileyladi:1
blue:32
sky:22
tortur:4
wkend:7
02:1
loung:10
jasonniad:1
shut:22
chri:10
johnson:4
cane:1
ask:87
class:82
regent:6
2day:24
aint:25
type:34
lmao:49
wi:6
h:21
twigga:2
bonfir:7
cooki:22
codysio:2
retweet:11
troystith:1
lunabee_art:1
realiti:16
jhillstephen:1
cherri:8
boyfriend:36
taho:2
texa:13
cw:2
celtic:4
flircr:1
total:83
snuggl:9
captainparish:1
sweet:95
clan:1
comic:5
con:6
troubl:15
lame:28
upshot:1
road:42
jilliankeen:1
countri:30
def:12
tomarrow:3
304:1
59:3
31:4
ecu:1
lotu:5
exig:1
terribl:31
shitti:9
andrew:6
kennedi:4
standup:1
comedi:5
brilliant

motorway:1
canker:1
onboard:2
kauan:1
yearbook:7
theoneeyedowl:1
hahahaha:15
vocal:3
jennifffaaaa:1
chillen:2
fcked:1
iamjonathancook:1
non:15
33:4
peanut:3
butter:8
wide:11
hd:8
sethsmaisi:1
karminabab:1
special:30
kitchen:13
effus:1
laney_jan:1
jessicaveronica:1
leren:1
heheh:16
toooooonight:1
debbiefletch:4
heyya:1
yes:15
mickfcknlovin:1
funeralsong:1
kim:11
deactiv:1
inkheart:1
altough:1
sassafrazza:1
certainli:3
employ:4
pirat:2
bay:9
17gy87:1
tpb:1
utfho:1
jewelri:6
maker:8
twib:1
7f8:1
chibiwong:1
famou:7
jimiyo:1
skim:2
mass:6
download:40
conniepray:1
spose:3
reach:16
102:2
followfriday:38
noeasybead:1
bead:4
bracelet:6
ash:8
coolest:6
begal:1
henri:4
hawon:1
firefly2020:1
reniewrit:1
reni:1
comfort:12
outdoor:6
loving_nkotb:1
alicewndrlnd:1
brunch:6
proceed:2
toast:13
bug:16
throw:13
sasukah:1
nicolewilson:1
sen:10
haileyatl:1
secret:12
tumblr:22
reheat:1
mountiehunt:1
oi:2
groupi:1
tobiasdewit:1
misschass:1
bawww:1
princess:11
bride:5
film:38
frshpriinc:2
sheep:5
danevo:1
gif

rucksack:2
larg:7
carrier:1
voddi:1
ciggi:1
earnt:1
kcortel:1
iannoyy:1
dumat:1
bezzi:1
tee_08:1
cozi:2
xo22n:1
laterss:1
amirasalvi:1
amira:1
bof:2
kenneth:1
copeland:1
hartson:1
bday:25
delta:3
dae:1
kiosk:1
centralworld:1
bleh:5
mhpalj:1
shake:15
400:5
beckym1985:1
babybear:1
fresh:13
luxemburg:1
acemas21:1
diahorrea:1
tinalouiseuk:1
sepponet:1
advic:12
glaad:1
medium:17
nokia:2
theatr:5
miishh:1
whore:2
imy2:1
chemic:2
mechan:3
pierc:6
veil:1
ukelel:1
mchechi:1
jedi1123:1
billwixey:1
creamer:1
vero:3
scenest:1
mrcav:1
monkey:8
wknd:9
relaxin:3
sympath:1
surgeri:11
amarie2209:1
scottwyden:1
scott:4
firework:4
lastnight:3
branden:1
namz66:1
doodlebugftdor:1
kobe:3
legit:7
dethron:1
inesgharsallah:1
surprisin:1
recogn:3
easili:7
ashley:9
tisdal:1
hporm:1
addthi:2
wowser:1
tweetnotpurge1:1
slay:1
aptli:1
mrssprousex:1
yar:1
tjdgafplz:1
39:2
cranberri:2
meloncamp:1
dlwilsonsworld:1
princ:7
spaghetti:2
graciou:1
msbeho:1
whoa:5
exclam:1
glambert:3
faith:4
bencusick92:1
nod:1
sah_rah:1
es

29th:2
chop:7
liver:4
tomato:7
tipiwel:1
4your:1
heidiheartshug:1
convinc:9
ink:6
batmobil:1
bkhh:1
afropunk:1
5th:7
egillhardar:1
tailgat:2
arancinibabi:1
_chacecrawford:1
fuelwrnchr:1
seowhow:1
sp:4
lia:1
travisgarland:1
swam:2
veronlynn:1
64:2
mjbthemvp:1
leafybear:1
ashley_elayn:1
gate:1
rod:3
coughlin:1
betk:1
sox:6
airr:1
dineshbabu:1
aarthycrazi:1
ubuntu:7
skype:10
lindsayk:1
fuuck:1
aughhhhhh:1
cinammon:1
decent:11
bitofmomsens:1
invert:1
nbw:1
monitor:9
japanes:7
huuum:1
yeaaaaah:1
littleburg:1
knw:2
spankin:1
dayum:2
pshhhht:1
rodrigomx:1
ohhhhh:1
tad:1
billder:1
mover:2
brunswick:1
youngest:2
outbreak:2
yeeeye:1
carolyn:1
6sffi:1
fuuuuuuuuuuuuuuuuuuuuuuuuck:1
saddddddd:1
weeek:1
dice:2
jacki:5
comedyqueen:2
portland:5
snub:1
weirdal:1
3ndh:1
villarr:1
champion:4
fucken:3
urghh:1
jink:1
kg86:2
punkpolkadot:2
deepxp:1
baxiabhishek:1
ceete:1
brajeshwar:1
preshit:1
comennt:1
afraid:10
slap:5
derri:1
buu:1
teter:1
tot:3
accent:1
toppu:1
mimi_loves_y:1
sarb:1
disast:5
swineflu:2
n

...

nathski:1
namroodinc:1
wokr:1
elinahafizah:1
fabulousssss:1
breckin:1
nph:1
joshuawith:2
bartercard:1
voucher:1
chilimel:1
christachi:1
aaaa:1
pada:1
abi:2
foto2:1
hxhx:1
kami:1
deh:1
kita:3
ngumpul:1
berencana:1
bikin:1
multipli:2
huhu:5
commerc:2
favorsbydorinda:1
dot12b:1
numpad:1
elissa_10807:1
moviee:1
marydotbox:1
5co4d:1
colon:1
spinelsun:1
austinaaron:1
krystynchong:1
y3nem:1
chrisbywoodium:1
philaskew:1
saruhh48:1
ikki_oo:1
marianasantos15:1
demetr:1
tumultu:1
leliedeann:1
congratz:1
pipolakii:1
katysmil:1
dcorsetto:1
pikshur:1
werki:1
gorillaglo:1
joel:2
nout:1
lissatsm:1
patkstereo:1
ciityliightsx3:1
iamthecommodor:1
djkaton:1
moesfalafel:1
jerzygrlli:1
blowi:1
wolferin:1
pennin:1
tomorra:1
benton:1
unplug:1
hellooooo:2
strongbow:1
4dx0m:1
toxaq:1
millennium:1
hazelnutpocki:1
diann:1
esthetician:1
masquerad:1
j_mccoy:1
telesal:1
50k:1
claudiachaouka1:1
icharlyy:1
beuati:1
czar:1
romanov:2
russia:2
cyberczar:1
johnmccain:1
mizzre:1
sammich:1
applecor:1
render:2
ooc:3
thegame1

glorifi:1
calcul:2
phe:1
dleightonmarissa:1
leighton:1
louisemurph:1
sushikat99:1
uah:1
peckhar:1
830:2
olerolerol:1
ndomingu:1
ddcrutch:1
sweetisa:1
blingylixi:1
beast:3
sourlullabi:1
virtual:2
congratualt:1
creami:2
soda:2
16th:2
amazin:2
farro:1
tink10270:1
yupyy:1
annd:1
dowl:1
jumpsuit:2
festivalish:1
michaelshatz:1
traumat:1
darkkarniv:1
powerpoint:1
sbjet:1
hr_minion:1
hrhappyhour:1
7lqbi:1
pzez8:1
julieroberts1:1
orangy68:1
offenc:1
presumptu:1
gumby33:1
segphault:1
nvidia:1
xorg:1
s98rl:1
layenni:1
union:2
tickl:2
ceciljohnson:1
offend:1
hip2nashvil:1
drift:1
86:2
kash18:1
ariherstand:1
firewir:1
kevinjwife57:1
2hr:1
waittt:1
megxcanxfli:1
missgreen:1
aat:1
loot:1
costco:1
tiffbasham58:1
sunnymchao:1
munchi:1
smanessa:1
cheekiest:1
2dabi:1
iambecomespanki:1
rtm9109:1
whirlwindmus:1
matthiaspatz:1
abcnew:1
lovefest:1
thon:1
be4:1
tcot:3
sgp:1
yankeeragu:1
raj_r:1
esukop:1
aureliom:2
mlomb:1
mrrodd:1
nooneai:1
yourmamaknow:1
leap:1
pepperoni:1
toaster:4
christian0386:1
yungpayn:

fangam:1
undisturb:1
writerluv3r:1
aaarrrggghhh:1
prelud:1
elud:1
sprang:1
6hea3:1
jazminerene:2
alexcurlyhair:1
tottenham:1
smp:1
tiller:1
blacktop:1
asheruk:1
latvianquiet:1
6f4i:1
sissi:2
gung:1
tking:1
tss:1
imon:1
missil:1
bigbigrabbit:1
jared_booth:1
a_montenegro:1
tbone:1
recap:1
fabio_z:1
digooooo:1
kitchencupboard:1
bycicl:1
dependend:1
doctorkarl:1
remark:1
gaudi:1
itsmesammond:1
andph112:1
amok_:1
tanmcg:1
tanya:1
lilshrimpit:1
bholbrook81:1
melissoua:1
dra6on:1
fairynuff:1
orchidflow:1
littleun:1
wahay:1
nomad:1
threshold:1
linkag:1
nbhgyg:1
endogoddess:1
cadburi:1
ladygaga:1
mikeyil:1
enuf:2
ricola:1
dayquil:1
blech:1
jayce_kay:1
adamlambertnew:1
tangerin:1
reinaxochitl:1
edjakyl:1
homebas:1
x9grw:1
krazyylegg:1
showgirl:1
somon:1
klyn1024:1
escourtn3i:1
agent_m:1
modernsinglemom:1
wilsonswar:1
exodu:1
1930:1
gsohandsom:1
hem:1
sonnyblack219:1
jensiii:1
mylifemyworld:1
moreilley:2
chad:2
6tj51:1
gabbyisact:1
dairuku:1
battlefield:1
bucknsj:1
scorer:1
danslevin:1
tart:1
mil

v242:1
chibicom:1
happybirthdaymik:1
mizsedz:1
69i5p:1
thaaank:1
saha:1
wrapper:1
zackrabbit:1
bizzar:2
diy:1
spacerocket:1
ohk:1
davidismyangel:1
catnam:1
2again:1
occaision:1
teacherpaddi:1
jaborwhalki:1
rudym55:1
fate:1
hooot:1
dropdeadjemila:1
overcom:1
hospitali:1
gronar:1
pleasantli:1
nonoysayz:1
malimili:1
exhust:1
vadi:1
ara:1
shannanstedman:1
hershey:2
moosegrind:1
supervann:1
ria:1
infidelnyc:1
welterweight:1
mini_mart:1
nathanfillion:1
dyou:1
thehyra:1
scroung:1
child0fthec:1
suuuuck:1
alwaysam:1
ib:1
electrcspacegrl:1
mistyp:1
philipbloom:1
5dvr8:1
princessopoulo:1
diner:1
peterborough:1
estat:1
2bumg:1
nessie_cullen10:1
charissenicol:1
sergeantgutt:1
mattmillr:1
gberd27:1
hqve:1
cliudi:1
vinoroma:1
xerox:1
5hs6j:1
frozentrous:1
tevez:1
5yr:1
manutd:1
realdollown:1
betti:1
leahtyp:1
hiteeni:1
camden:1
knightsbridg:1
oookaay:1
h0llyhart:1
dill:1
lilnana:1
vh1renchi:1
saaphyri:1
gellar:1
andygozo:1
useless:3
thatdop:1
biznez:1
xaldin:1
cdashmoney:1
roadi:1
7io7:1
huntsvil:1
r

incrimin:1
ldvkz:1
sukaaaa:1
jadi:1
mati:1
donor:1
jantung:1
demoon39:1
lozzaybaybay:1
lazay:1
baybay:1
hbu:1
tobyclick:1
belgium:2
acces:1
dsthestar1121:1
jblovatogomez:1
ahha:1
rochelleveturi:1
michalabana:1
ticketek:1
drgelb:1
holidayi:1
yousuckdv:1
loverr:1
botdfmus:1
sorryy:1
playstat:2
crazyy:1
stalin:1
ratee:1
thinkinn:1
yeww:1
sunfeast:1
lore:1
guna:1
truantrebellion:1
blatant:1
balto:1
suckafreebrie:1
graced:1
deanwilliam:1
chillaxingggg:1
shannynb:1
wce:1
rivet:1
shepherd:2
jamieleecart:1
fowt:1
adoran2:1
wolvesatnight:1
uo:1
tweethug:2
crow74:2
amybabydoll69:1
julia_13:1
jerevien:1
maddow:1
z463o:1
riblah:1
shanelain:1
canteen:1
hellenbach:1
geordi:2
lagunita:1
sooooooooooon:1
orthodontist:1
lilmellie71:1
trucker:1
debdoyl:1
nottingham:1
ernstjanb:1
leaseweb:1
notintofashion:1
pena:1
dele:1
wahahaha:1
joeyselena:1
somuch:1
fleec:1
trusti:1
tm:1
reimer:1
webdude75:1
beachbodi:1
betterbodi:1
betterbody90:1
avaragent901:1
jeffco:1
letd:1
aligraysanatomi:1
footankl:1
toddjg:1
he

In [7]:
fdist.plot(20, cumulative=False)

<Figure size 640x480 with 1 Axes>

In [8]:
from heapq import nlargest 

Five_k_Highest = nlargest(5000, fdist, key = fdist.get) 

In [9]:
len(Five_k_Highest)

5000

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:
# your code here

features= []

for tag,sentence in zip(sentiment_sample['target'],sentiment_sample['text_processed']):
    word_dict={}
    for word in sentence:
        if word in Five_k_Highest :
            word_dict[word]=True
        else:
            word_dict[word]=False
    features.append((word_dict,tag))

In [11]:
features

[({'miss': True,
   'nikki': True,
   'nu': True,
   'alreadi': True,
   'alway': True,
   'need': True,
   'thank': True,
   'u': True,
   'xxx': True},
  0),
 ({'dream': True,
   'last': True,
   'night': True,
   'rememb': True,
   'sign': True,
   'clearli': True,
   'told': True,
   'get': True,
   'job': True,
   'remem': False,
   'said': True},
  0),
 ({'girlyghost': False,
   'ohh': True,
   'poor': True,
   'sickli': False,
   'hug': True,
   'hope': True,
   'feel': True,
   'littl': True,
   'better': True,
   'soon': True},
  0),
 ({'rain': True}, 0),
 ({'misskeribabi': False, 'wish': True, 'wa': True, 'la': True, 'right': True},
  0),
 ({'nala': False,
   'olowalu': False,
   'still': True,
   'ha': True,
   'full': True,
   'tummi': True,
   'bread': True,
   'basket': True,
   '2': True,
   'http': True,
   'app': True,
   'facebook': True,
   'com': True,
   'dogbook': True,
   'profil': True,
   'view': True,
   '6285546': False},
  0),
 ({'macintom': False,
   'site'

In [12]:
sentiment_sample['target'].value_counts()


4    10049
0     9951
Name: target, dtype: int64

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [13]:
# your code here
from sklearn.model_selection import train_test_split

sentiment_sample_train, sentiment_sample_test = train_test_split(features, test_size=0.20)


In [14]:
classifier = nltk.NaiveBayesClassifier.train(sentiment_sample_train)
classifier.show_most_informative_features()

Most Informative Features
            followfriday = True                4 : 0      =     20.3 : 1.0
                     sad = True                0 : 4      =     18.1 : 1.0
                 depress = True                0 : 4      =     17.7 : 1.0
                   fever = True                0 : 4      =     15.0 : 1.0
                    knee = True                0 : 4      =     13.7 : 1.0
                    lone = True                0 : 4      =     13.7 : 1.0
               congratul = True                4 : 0      =     13.6 : 1.0
                  cancel = True                0 : 4      =     13.0 : 1.0
                     rip = True                0 : 4      =     11.0 : 1.0
                    neck = True                0 : 4      =     11.0 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [15]:
# your code here
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, sentiment_sample_test))*100)

classifier.show_most_informative_features(15)

Classifier accuracy percent: 72.0
Most Informative Features
            followfriday = True                4 : 0      =     20.3 : 1.0
                     sad = True                0 : 4      =     18.1 : 1.0
                 depress = True                0 : 4      =     17.7 : 1.0
                   fever = True                0 : 4      =     15.0 : 1.0
                    knee = True                0 : 4      =     13.7 : 1.0
                    lone = True                0 : 4      =     13.7 : 1.0
               congratul = True                4 : 0      =     13.6 : 1.0
                  cancel = True                0 : 4      =     13.0 : 1.0
                     rip = True                0 : 4      =     11.0 : 1.0
                    neck = True                0 : 4      =     11.0 : 1.0
                  welcom = True                4 : 0      =     10.7 : 1.0
                  throat = True                0 : 4      =     10.2 : 1.0
                     fml = True         

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [16]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [17]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [18]:
# your code here